In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Projects\PrognosAI\dataset\train_FD001.txt", sep=r"\s+", header=None)


In [3]:
column_names = ['engine_id', 'cycle'] + \
               [f'op_setting_{i}' for i in range(1, 4)] + \
               [f'sensor_{i}' for i in range(1, 22)]

df.columns = column_names

In [9]:
df.groupby('engine_id')['cycle'].count()


engine_id
1      192
2      287
3      179
4      189
5      269
      ... 
96     336
97     202
98     156
99     185
100    200
Name: cycle, Length: 100, dtype: int64

In [10]:
df.groupby('engine_id')['cycle'].max()


engine_id
1      192
2      287
3      179
4      189
5      269
      ... 
96     336
97     202
98     156
99     185
100    200
Name: cycle, Length: 100, dtype: int64

In [11]:
df.groupby('engine_id').engine_id.count()


engine_id
1      192
2      287
3      179
4      189
5      269
      ... 
96     336
97     202
98     156
99     185
100    200
Name: engine_id, Length: 100, dtype: int64

In [12]:
df.groupby('engine_id')['sensor_2'].max()


engine_id
1      644.21
2      643.94
3      643.93
4      644.53
5      644.02
        ...  
96     644.20
97     643.97
98     644.39
99     644.10
100    643.95
Name: sensor_2, Length: 100, dtype: float64

In [13]:
df.groupby('engine_id')['sensor_7'].min()


engine_id
1      550.70
2      550.85
3      551.31
4      551.60
5      550.77
        ...  
96     550.78
97     551.55
98     551.27
99     550.64
100    550.68
Name: sensor_7, Length: 100, dtype: float64

In [14]:
df.groupby('engine_id')['cycle'].agg(['min', 'max', 'count'])


,min,max,count
engine_id,,,
1,1,192,192
2,1,287,287
3,1,179,179
4,1,189,189
5,1,269,269
...,...,...,...
96,1,336,336
97,1,202,202
98,1,156,156


In [15]:
df.groupby('engine_id')['sensor_2'].agg(['mean', 'min', 'max', 'std'])


,mean,min,max,std
engine_id,,,,
1,642.621042,641.71,644.21,0.486795
2,642.435226,641.27,643.94,0.560214
3,642.543743,641.48,643.93,0.474190
4,642.662381,641.81,644.53,0.441446
5,642.451970,641.30,644.02,0.551542
...,...,...,...,...
96,642.629821,641.55,644.20,0.501021
97,642.522624,641.50,643.97,0.473922
98,642.584872,641.62,644.39,0.525905


In [16]:
df.groupby(['engine_id', 'cycle'])['sensor_2'].mean()


engine_id  cycle
1          1        641.82
           2        642.15
           3        642.35
           4        642.35
           5        642.37
                     ...  
100        196      643.49
           197      643.54
           198      643.42
           199      643.23
           200      643.85
Name: sensor_2, Length: 20631, dtype: float64

In [17]:
type(df.groupby(['engine_id', 'cycle']).size().index)


pandas.core.indexes.multi.MultiIndex

In [18]:
grouped = df.groupby(['engine_id', 'cycle'])['sensor_2'].mean()
grouped = grouped.reset_index()


In [19]:
max_cycle = df.groupby('engine_id')['cycle'].max()

df = df.merge(max_cycle, on='engine_id', suffixes=('', '_max'))
df['RUL'] = df['cycle_max'] - df['cycle']


In [20]:
engine_life = df.groupby('engine_id')['cycle'].max().reset_index()
engine_life.sort_values(by='cycle', ascending=False)


,engine_id,cycle
68,69,362
91,92,341
95,96,336
66,67,313
82,83,293
...,...,...
57,58,147
69,70,137
56,57,137
90,91,135


In [21]:
df.sort_values(by='sensor_2', ascending=False).head()


,engine_id,cycle,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,cycle_max,RUL
846,4,189,-0.0024,-0.0002,100.0,518.67,644.53,1612.11,1432.55,14.62,...,8259.42,8.5246,0.03,398,2388,100.0,38.44,23.1006,189,0
7636,39,127,0.0004,0.0001,100.0,518.67,644.53,1595.03,1426.95,14.62,...,8105.83,8.5383,0.03,395,2388,100.0,38.29,23.1090,128,1
840,4,183,0.0034,0.0002,100.0,518.67,644.50,1602.97,1426.55,14.62,...,8239.91,8.5034,0.03,399,2388,100.0,38.47,23.0511,189,6
3303,16,208,0.0024,-0.0002,100.0,518.67,644.47,1595.68,1437.53,14.62,...,8137.23,8.5419,0.03,396,2388,100.0,38.44,23.0087,209,1
9050,46,256,0.0023,0.0002,100.0,518.67,644.44,1602.57,1433.99,14.62,...,8118.01,8.5052,0.03,397,2388,100.0,38.38,23.0170,256,0


In [22]:
engine_data = df.groupby('engine_id')

degradation = engine_data['sensor_2'].agg(['first', 'last'])
degradation['change'] = degradation['last'] - degradation['first']


In [23]:
df.groupby('cycle')['sensor_2'].mean()


cycle
1      642.3972
2      642.3483
3      642.4103
4      642.4022
5      642.3497
         ...   
358    643.7400
359    643.4600
360    643.9700
361    643.6500
362    643.7500
Name: sensor_2, Length: 362, dtype: float64